<a href="https://colab.research.google.com/github/sajal1302/Study-Material/blob/master/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 47.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=96a7e31d3bdebeb3b3abcaa260d1b255b0950a4a621568c017559f57273f9a13
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [3]:
def get_spark_session():
    spark = SparkSession.builder.appName("BCG-Project").enableHiveSupport().getOrCreate()
    return spark

In [5]:
config = { 'files' : ['Charges_use.csv' ,
                      'Damages_use.csv' ,
                      'Endorse_use.csv',
                      'Primary_Person_use.csv' ,
                      'Restrict_use.csv' ,
                      'Units_use.csv'] ,
          'files_loc' : '',
          
          'feeds':{
              'Charges_use': {
                  'feed_name' : 'Charges_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Damages_use': {
                  'feed_name' : 'Damages_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Endorse_use': {
                  'feed_name' : 'Damages_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Primary_Person_use': {
                  'feed_name' : 'Primary_Person_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Restrict_use': {
                  'feed_name' : 'Restrict_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Units_use': {
                  'feed_name' : 'Units_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              }
          }  
    
}

In [6]:
spark = get_spark_session()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
spark

In [15]:
! ls drive/MyDrive/bcg/staging_files/

Charges_use.csv  Endorse_use.csv	 Restrict_use.csv
Damages_use.csv  Primary_Person_use.csv  Units_use.csv


In [20]:
! ls bcg/config

config.json


In [21]:
import json

In [28]:
def load_json(file_path):
  with open(file_path) as f:
    config = json.load(f)
    return config

In [31]:
bcg_conf = load_json('bcg/config/config.json')

In [32]:
bcg_conf['feeds']

{'feeds': {'Charges_use': {'compression': '',
   'delimeter': '',
   'feed_location': 'bcg/staging_files/',
   'feed_name': 'Charges_use.csv'},
  'Damages_use': {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Damages_use.csv'},
  'Endorse_use': {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Damages_use.csv'},
  'Primary_Person_use': {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Primary_Person_use.csv'},
  'Restrict_use': {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Restrict_use.csv'},
  'Units_use': {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Units_use.csv'}}}

In [35]:
bcg_conf['feeds'].items()

dict_items([('Charges_use', {'feed_name': 'Charges_use.csv', 'feed_location': 'bcg/staging_files/', 'delimeter': '', 'compression': ''}), ('Damages_use', {'feed_name': 'Damages_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''}), ('Endorse_use', {'feed_name': 'Damages_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''}), ('Primary_Person_use', {'feed_name': 'Primary_Person_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''}), ('Restrict_use', {'feed_name': 'Restrict_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''}), ('Units_use', {'feed_name': 'Units_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''})])

In [48]:
for a in bcg_conf['feeds']:
  print(a, bcg_conf['feeds'][a], bcg_conf['feeds'][a]['feed_name'],  bcg_conf['feeds'][a]['feed_location']  )

Charges_use {'feed_name': 'Charges_use.csv', 'feed_location': 'bcg/staging_files/', 'delimeter': '', 'compression': ''} Charges_use.csv bcg/staging_files/
Damages_use {'feed_name': 'Damages_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''} Damages_use.csv 
Endorse_use {'feed_name': 'Damages_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''} Damages_use.csv 
Primary_Person_use {'feed_name': 'Primary_Person_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''} Primary_Person_use.csv 
Restrict_use {'feed_name': 'Restrict_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''} Restrict_use.csv 
Units_use {'feed_name': 'Units_use.csv', 'feed_location': '', 'delimeter': '', 'compression': ''} Units_use.csv 


In [39]:
feed_dict = list(bcg_conf['feeds'].items())

In [43]:
feed_dict

[('Charges_use',
  {'compression': '',
   'delimeter': '',
   'feed_location': 'bcg/staging_files/',
   'feed_name': 'Charges_use.csv'}),
 ('Damages_use',
  {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Damages_use.csv'}),
 ('Endorse_use',
  {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Damages_use.csv'}),
 ('Primary_Person_use',
  {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Primary_Person_use.csv'}),
 ('Restrict_use',
  {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Restrict_use.csv'}),
 ('Units_use',
  {'compression': '',
   'delimeter': '',
   'feed_location': '',
   'feed_name': 'Units_use.csv'})]

In [40]:
for feed, value in feed_dict[0] :
  print(feed , feed_dict[feed]['feed_name'])


ValueError: ignored